### Import Packages

In [1]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
from requests.utils import requote_uri
from fake_useragent import UserAgent
from lxml import html
import random
import time
from selenium import webdriver

### Process Data

In [10]:
FILE = 'urldump.xlsx'
url_dump = pd.read_excel(FILE, sheet_name='Sheet1')
url_dump.head()

,ID,Naam,Provincie,Website,Country
0,6bf8485d-29f5-e711-811e-e0071b652f71,ARTOOS,Vlaams-Brabant,https://www.artoos.com,Belgium
1,eb1676b1-9796-e611-80f3-5065f38b1581,AUTOMATION,Vlaams-Brabant,https://www.automation.be,Belgium
2,1454d150-d778-e611-80ef-5065f38b1581,BIOTECHNOLOGICAL ENZYMATIC CATALYSE,Oost-Vlaanderen,http://www.bienca.com,Belgium
3,29eda1f5-842c-e611-815c-c4346bad60c4,DE SUTTER,Vlaams-Brabant,https://www.desutter-naturally.com,Belgium
4,87296532-fd6d-e611-80e5-5065f38bc5a1,DS-WIND,Limburg,NaN,Belgium


In [18]:
url_missing = url_dump[url_dump['Website'].isnull()]
url_missing.head()

,ID,Naam,Provincie,Website,Country
4,87296532-fd6d-e611-80e5-5065f38bc5a1,DS-WIND,Limburg,NaN,Belgium
9,4f3430c7-2af5-e711-811e-e0071b652f71,LARES,Limburg,NaN,Belgium
16,830d10be-2bf5-e711-8126-e0071b65be81,VANNEELS,Antwerpen,NaN,Belgium
19,458ead00-29f5-e711-8126-e0071b65be81,!Nspire,Limburg,NaN,Belgium
22,00233b99-1928-eb11-a813-0022489bb663,"""Delcampo""; ""Delcampo Creative Consulting”; “V...",Antwerpen,NaN,Belgium


In [14]:
url_not_missing = url_dump[url_dump['Website'].notnull()]
url_not_missing.head()

,ID,Naam,Provincie,Website,Country
0,6bf8485d-29f5-e711-811e-e0071b652f71,ARTOOS,Vlaams-Brabant,https://www.artoos.com,Belgium
1,eb1676b1-9796-e611-80f3-5065f38b1581,AUTOMATION,Vlaams-Brabant,https://www.automation.be,Belgium
2,1454d150-d778-e611-80ef-5065f38b1581,BIOTECHNOLOGICAL ENZYMATIC CATALYSE,Oost-Vlaanderen,http://www.bienca.com,Belgium
3,29eda1f5-842c-e611-815c-c4346bad60c4,DE SUTTER,Vlaams-Brabant,https://www.desutter-naturally.com,Belgium
5,feb85200-34f5-e711-816d-e0071b66d0e1,EDDY MERCKX CYCLES,Vlaams-Brabant,https://www.eddymerckx.com,Belgium


In [20]:
url_missing.insert(5, "Google_Result", "") 
url_missing.insert(6, "Bing_Result", "") 
url_not_missing.insert(5, "Google_Result", "") 
url_not_missing.insert(6, "Bing_Result", "") 

In [ ]:
url_missing = url_missing.sample(n=300, random_state=123)
url_not_missing = url_not_missing.sample(n=300, random_state=123)

In [25]:
#Save in CSV files
url_missing.to_csv('missing_url_random_300.csv', index=False, sep=';')
url_not_missing.to_csv('not_missing_url_random_300.csv', index=False, sep=';')

In [ ]:
#Load CSV files
url_missing = pd.read_csv('missing_url_random_300.csv', sep=';')
url_not_missing = pd.read_csv('not_missing_url_random_300.csv', sep=';')

### Scrape Missing URLs

In [43]:
Counter = 0
start = time.time()

browser = webdriver.Chrome('C:\\Users\\nusret\\Desktop\\Scrapping\\Drivers\\chromedriver_89')
browser.get('https://www.google.be/search?q=start')
time.sleep(30)

for index, row in url_missing[221:].iterrows():

    #Search Google
    Base_URL = requote_uri('https://www.google.be/search?q=' + row['Naam'] + ' bedrijf' + '&gws_rd=cr')
    input = browser.find_element_by_name('q')
    input.clear()
    time.sleep(random.randint(1,3))
    input.send_keys(row['Naam'])
    time.sleep(random.randint(2,5))
    try:
        browser.find_element_by_css_selector("#tsf > div:nth-child(2) > div.A8SBwf.emcav > div.RNNXgb > button").click()
    except:
        browser.get(Base_URL)
        
    soup = BeautifulSoup(browser.page_source)
    try:
        url_missing.loc[index, 'Google_Result'] = soup.select('#rhs > div > div.kp-blk.knowledge-panel.Wnoohf.OJXvsb > div > div.ifM9O > div > div.kp-header > div.fYOrjf.kp-hc > div.Hhmu2e.wDYxhc.NFQFxe.viOShc.LKPcQc > div > div > div > div:nth-child(1) > a')[0]['href']
    except:
        url_missing.loc[index, 'Google_Result'] = soup.findAll("cite")[0].text.split('›')[0].rstrip()        
    
    time.sleep(random.randint(3,5))
    #Search Bing
    Base_URL = requote_uri('https://www.bing.com/search?cc=be&q=' + row['Naam'] + ' bedrijf')
    browser.get(Base_URL)
    time.sleep(random.randint(1,3))
    soup = BeautifulSoup(browser.page_source)
    try:
        url_missing.loc[index, 'Bing_Result'] = soup.select('#b_context > li > div.b_entityTP > div:nth-child(2) > div > div > div:nth-child(2) > div > div.b_icbtns > div > div > a')[0]['href']
    except:
        url_missing.loc[index, 'Bing_Result'] = soup.findAll("cite")[0].text.split('›')[0].rstrip()    
   
    if ((Counter % 10) == 0):
        end = time.time()
        print('Iteration Number ' + str(Counter) + ' Completed - Total Elapsed Time: %.02f minutes' % ((end - start)/60))
    Counter += 1
    time.sleep(random.randint(30,38))

Iteration Number 230 Completed - Total Elapsed Time: 8.17 minutes
Iteration Number 240 Completed - Total Elapsed Time: 16.12 minutes
Iteration Number 250 Completed - Total Elapsed Time: 24.02 minutes
Iteration Number 260 Completed - Total Elapsed Time: 32.15 minutes
Iteration Number 270 Completed - Total Elapsed Time: 39.66 minutes
Iteration Number 280 Completed - Total Elapsed Time: 47.42 minutes
Iteration Number 290 Completed - Total Elapsed Time: 55.32 minutes


In [16]:
#Save scraped data
url_missing.to_excel("url_missing_random_300_scraped_excel.xlsx", sheet_name='Sheet_name_1')  

### Scrape Non Missing URLs

In [10]:
Counter = 0
start = time.time()

browser = webdriver.Chrome('C:\\Users\\nusret\\Desktop\\Scrapping\\Drivers\\chromedriver_89')
browser.get('https://www.google.be/search?q=start')
time.sleep(30)

for index, row in url_not_missing[178:].iterrows():

    #Search Google
    Base_URL = requote_uri('https://www.google.be/search?q=' + row['Naam'] + ' bedrijf' + '&gws_rd=cr')
    input = browser.find_element_by_name('q')
    input.clear()
    time.sleep(random.randint(1,3))
    input.send_keys(row['Naam'])
    time.sleep(random.randint(2,5))
    try:
        browser.find_element_by_css_selector("#tsf > div:nth-child(2) > div.A8SBwf.emcav > div.RNNXgb > button").click()
    except:
        browser.get(Base_URL)
        
    soup = BeautifulSoup(browser.page_source)
    try:
        url_not_missing.loc[index, 'Google_Result'] = soup.select('#rhs > div > div.kp-blk.knowledge-panel.Wnoohf.OJXvsb > div > div.ifM9O > div > div.kp-header > div.fYOrjf.kp-hc > div.Hhmu2e.wDYxhc.NFQFxe.viOShc.LKPcQc > div > div > div > div:nth-child(1) > a')[0]['href']
    except:
        url_not_missing.loc[index, 'Google_Result'] = soup.findAll("cite")[0].text.split('›')[0].rstrip()        
    
    time.sleep(random.randint(3,5))
    #Search Bing
    Base_URL = requote_uri('https://www.bing.com/search?cc=be&q=' + row['Naam'] + ' bedrijf')
    browser.get(Base_URL)
    time.sleep(random.randint(1,3))
    soup = BeautifulSoup(browser.page_source)
    try:
        url_not_missing.loc[index, 'Bing_Result'] = soup.select('#b_context > li > div.b_entityTP > div:nth-child(2) > div > div > div:nth-child(2) > div > div.b_icbtns > div > div > a')[0]['href']
    except:
        url_not_missing.loc[index, 'Bing_Result'] = soup.findAll("cite")[0].text.split('›')[0].rstrip()    
   
    if ((Counter % 10) == 0):
        end = time.time()
        print('Iteration Number ' + str(Counter) + ' Completed - Total Elapsed Time: %.02f minutes' % ((end - start)/60))
    Counter += 1
    time.sleep(random.randint(30,38))

Iteration Number 0 Completed - Total Elapsed Time: 0.81 minutes
Iteration Number 10 Completed - Total Elapsed Time: 9.21 minutes
Iteration Number 20 Completed - Total Elapsed Time: 16.80 minutes
Iteration Number 30 Completed - Total Elapsed Time: 24.75 minutes
Iteration Number 40 Completed - Total Elapsed Time: 32.82 minutes
Iteration Number 50 Completed - Total Elapsed Time: 40.87 minutes
Iteration Number 60 Completed - Total Elapsed Time: 48.96 minutes
Iteration Number 70 Completed - Total Elapsed Time: 56.98 minutes
Iteration Number 80 Completed - Total Elapsed Time: 65.29 minutes
Iteration Number 90 Completed - Total Elapsed Time: 73.57 minutes
Iteration Number 100 Completed - Total Elapsed Time: 81.89 minutes
Iteration Number 110 Completed - Total Elapsed Time: 90.04 minutes
Iteration Number 120 Completed - Total Elapsed Time: 98.23 minutes


In [ ]:
#Save scraped data
url_not_missing.to_excel("url_not_missing_random_300_scraped_excel.xlsx", sheet_name='Sheet_name_1')  